In [1]:
import pandas as pd
import numpy as np
from utils import *
import pytensor.tensor as pt  # Import PyTensor (Theano backend)

In [2]:
df = read_dd_data('../data/PD_data')


In [3]:
df

,rcert,runcert,event_prob,choice,condition,rt,odds,participant
0,5.0,9.50,0.10,0.0,1.0,2013.5,9.000000,bh348lli7
1,10.0,11.11,0.90,0.0,1.0,514.4,0.111111,bh348lli7
2,1.0,1.10,0.50,0.0,1.0,424.2,1.000000,bh348lli7
3,1.0,31.00,0.25,0.0,1.0,398.2,3.000000,bh348lli7
4,20.0,80.00,0.50,0.0,1.0,538.3,1.000000,bh348lli7
...,...,...,...,...,...,...,...,...
9795,-10.0,-100.00,0.10,0.0,2.0,1504.1,9.000000,u3yyfob1i
9796,-10.0,-40.00,0.25,1.0,2.0,1856.4,3.000000,u3yyfob1i
9797,-5.0,-9.50,0.10,1.0,2.0,1911.6,9.000000,u3yyfob1i
9798,-50.0,-100.00,0.75,0.0,2.0,2848.4,0.333333,u3yyfob1i


In [4]:
df.describe()

,rcert,runcert,event_prob,choice,condition,rt,odds
count,9749.000000,9749.000000,9749.000000,9749.000000,9749.000000,9749.000000,9749.000000
mean,0.068212,0.385589,0.500164,0.393887,1.498513,2513.978336,2.690339
std,24.579049,562.935370,0.298548,0.488635,0.500023,1438.240239,3.318231
min,-50.000000,-4550.000000,0.100000,0.000000,1.000000,200.000000,0.111111
25%,-10.000000,-28.000000,0.250000,0.000000,1.000000,1545.400000,0.333333
50%,1.000000,1.010000,0.500000,0.000000,1.000000,2185.600000,1.000000
75%,10.000000,31.000000,0.750000,1.000000,2.000000,3141.000000,3.000000
max,50.000000,4550.000000,0.900000,1.000000,2.000000,9929.200000,9.000000


In [5]:
print(df.isna().sum())  # Check for NaNs

rcert          0
runcert        0
event_prob     0
choice         0
condition      0
rt             0
odds           0
participant    0
dtype: int64


In [6]:
# Extract participant information
participants = df["participant"].unique()
n_participants = len(participants)

# Convert participant IDs to integer indices
participant_idx = {pid: i for i, pid in enumerate(participants)}
df["participant_idx"] = df["participant"].map(participant_idx)

'''
Because of the different levels of magnitude
we decide to standarize rewards and reaction times
'''

# used later for unscaling
scaling_factors = {
    "rt_mean": df['rt'].mean(), "rt_std": df['rt'].std(),
    "rcert_mean": df['rcert'].mean(), "rcert_std": df['rcert'].std(),
    "runcert_mean": df['runcert'].mean(), "runcert_std": df['runcert'].std()
}

df["rcert"] = (df["rcert"] - scaling_factors["rcert_mean"]) / scaling_factors["rcert_std"]
df["runcert"] = (df["runcert"] - scaling_factors["runcert_mean"]) / scaling_factors["runcert_std"]
df["rt"] = (df["rt"] - scaling_factors["rt_mean"]) / scaling_factors["rt_std"]

df

,rcert,runcert,event_prob,choice,condition,rt,odds,participant,participant_idx
0,0.200650,0.016191,0.10,0.0,1.0,-0.347980,9.000000,bh348lli7,0
1,0.404075,0.019051,0.90,0.0,1.0,-1.390295,0.111111,bh348lli7,0
2,0.037910,0.001269,0.50,0.0,1.0,-1.453011,1.000000,bh348lli7,0
3,0.037910,0.054384,0.25,0.0,1.0,-1.471088,3.000000,bh348lli7,0
4,0.810926,0.141427,0.50,0.0,1.0,-1.373678,1.000000,bh348lli7,0
...,...,...,...,...,...,...,...,...,...
9795,-0.409626,-0.178325,0.10,0.0,2.0,-0.702162,9.000000,u3yyfob1i,48
9796,-0.409626,-0.071741,0.25,1.0,2.0,-0.457210,3.000000,u3yyfob1i,48
9797,-0.206201,-0.017561,0.10,1.0,2.0,-0.418830,9.000000,u3yyfob1i,48
9798,-2.037028,-0.178325,0.75,0.0,2.0,0.232521,0.333333,u3yyfob1i,48


In [7]:
# Keep only the first participant
# first_participant = participants[0]
# df = df[df['participant'] == first_participant]

# Display first row to verify
df


,rcert,runcert,event_prob,choice,condition,rt,odds,participant,participant_idx
0,0.200650,0.016191,0.10,0.0,1.0,-0.347980,9.000000,bh348lli7,0
1,0.404075,0.019051,0.90,0.0,1.0,-1.390295,0.111111,bh348lli7,0
2,0.037910,0.001269,0.50,0.0,1.0,-1.453011,1.000000,bh348lli7,0
3,0.037910,0.054384,0.25,0.0,1.0,-1.471088,3.000000,bh348lli7,0
4,0.810926,0.141427,0.50,0.0,1.0,-1.373678,1.000000,bh348lli7,0
...,...,...,...,...,...,...,...,...,...
9795,-0.409626,-0.178325,0.10,0.0,2.0,-0.702162,9.000000,u3yyfob1i,48
9796,-0.409626,-0.071741,0.25,1.0,2.0,-0.457210,3.000000,u3yyfob1i,48
9797,-0.206201,-0.017561,0.10,1.0,2.0,-0.418830,9.000000,u3yyfob1i,48
9798,-2.037028,-0.178325,0.75,0.0,2.0,0.232521,0.333333,u3yyfob1i,48


In [8]:
'''
TODOS:

- loss ul fluctueaza de la 50 la 3e+11.
'''

'\nTODOS:\n\n- we get a loss of 3e+11. Something is going in the opposite direction.\n'

In [9]:
# Extract observed data
rcert = df["rcert"].values
runcert = df["runcert"].values
event_prob = df["event_prob"].values
choice_data = df["choice"].astype(int).values  # Ensure it's an integer for Bernoulli
rt_data = df["rt"].values
participant_ids = df["participant_idx"].values  # Integer IDs

# Define hierarchical PyMC model
with pm.Model() as model:
    # Group-level priors
    mu_k = pm.Normal("mu_k", mu=0, sigma=1)  
    sigma_k = pm.HalfNormal("sigma_k", sigma=1)  
    
    # beta i.e. sensitivity needs to be strictly positive
    mu_beta = pm.HalfNormal("mu_beta", sigma=1)
    sigma_beta = pm.HalfNormal("sigma_beta", sigma=1)

    # since we work with normalized reaction times this can be negative as well
    mu_beta0 = pm.Normal("mu_beta0", mu=0, sigma=1)
    sigma_beta0 = pm.HalfNormal("sigma_beta0", sigma=1)

    # beta1 i.e. sensitivity needs to be strictly positive
    mu_beta1 = pm.HalfNormal("mu_beta1", sigma=1)
    sigma_beta1 = pm.HalfNormal("sigma_beta1", sigma=1)

    # Participant-level parameters (hierarchical structure)
    k = pm.Normal("k", mu=mu_k, sigma=sigma_k, shape=n_participants)
    beta = pm.LogNormal("beta", mu=mu_beta, sigma=sigma_beta, shape=n_participants)
    beta0 = pm.Normal("beta0", mu=mu_beta0, sigma=sigma_beta0, shape=n_participants)
    beta1 = pm.LogNormal("beta1", mu=mu_beta1, sigma=sigma_beta1, shape=n_participants)

    # Compute subjective values using hyperbolic discounting
    SV_certain = rcert
    SV_uncertain = runcert / (1 + k[participant_ids] * event_prob)

    # Choice model (Bernoulli with logistic function)
    max_exp = 10
    # TODO: slide 36 of lecture 4 she uses the oposite difference
    exp = beta[participant_ids] * (SV_uncertain - SV_certain) # TODO: check order of SV_uncertain and SV_certain
    exp = pm.math.clip(exp, -max_exp, max_exp)

    prob_choose_uncertain = pm.Deterministic("prob_choose_uncertain",
        pm.math.sigmoid(exp) 
    )
    choice = pm.Bernoulli("choice", p=prob_choose_uncertain, observed=choice_data)

    # Reaction time model (log-normal)
    decision_difficulty = (SV_uncertain - SV_certain) ** 2
    mu_RT = pm.Deterministic("mu_RT", beta0[participant_ids] + beta1[participant_ids] * decision_difficulty)
    # NOTE: remember we are using log RTs not the actual RTs
    log_RT = pm.Normal("log_RT", mu=mu_RT, sigma=0.3, observed=rt_data)

    model.debug(verbose=True)

    # Run sampling
    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True, init='advi')

# Save trace to disk
trace.to_netcdf("probabilistic_discounting_trace.nc")


point={'mu_k': array(0.), 'sigma_k_log__': array(0.), 'mu_beta_log__': array(0.), 'sigma_beta_log__': array(0.), 'mu_beta0': array(0.), 'sigma_beta0_log__': array(0.), 'mu_beta1_log__': array(0.), 'sigma_beta1_log__': array(0.), 'k': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'beta_log__': array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5,
       1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5,
       1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5,
       1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5]), 'beta0': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'beta1_log__': array([1.5, 1.5

Initializing NUTS using advi...


Output()

Convergence achieved at 82500
Interrupted at 82,499 [41%]: Average Loss = 4.0865e+19
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_k, sigma_k, mu_beta, sigma_beta, mu_beta0, sigma_beta0, mu_beta1, sigma_beta1, k, beta, beta0, beta1]


Output()

ValueError: Not enough samples to build a trace.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt

# Load the saved trace from disk
trace = az.from_netcdf("probabilistic_discounting_trace.nc")


# Trace plot: Check parameter convergence across chains
az.plot_trace(trace, var_names=["mu_k", "mu_beta", "mu_beta0", "mu_beta1"])

plt.show()

# Posterior distributions: Check inferred parameter values
az.plot_posterior(trace, var_names=["mu_k", "mu_beta", "mu_beta0", "mu_beta1"])
plt.show()

# Check R-hat values: Should be close to 1.00 for good convergence
rhat_values = az.rhat(trace)
print("R-hat values (should be ~1.00):\n", rhat_values)

# Check effective sample size (ESS): Should be > 200 per chain
ess_values = az.ess(trace)
print("Effective Sample Size (ESS) per parameter:\n", ess_values)

# Energy plot: Helps diagnose divergences
az.plot_energy(trace)
plt.show()


# Pair plot for key parameters (shows correlation structure)
az.plot_pair(trace, var_names=["mu_k", "mu_beta", "mu_beta0", "mu_beta1"], kind="scatter", divergences=True)
plt.show()

In [ ]:
def predict_participant_behavior(participant_id, trace, data):
    """
    Predict reaction time and choice for a given participant based on model parameters.
    
    Parameters:
    -----------
    participant_id : str
        The participant ID to make predictions for
    trace : arviz.InferenceData
        The MCMC trace containing parameter posterior samples
    data : pd.DataFrame
        The original data containing trial information
        
    Returns:
    --------
    pd.DataFrame
        Dataframe with predicted RTs and choices for the participant's trials

        TODO: not working yet because we only have one participant
    """
    # Get participant index
    participant_data = data[data['participant'] == participant_id]
    if len(participant_data) == 0:
        raise ValueError(f"Participant {participant_id} not found in data")
    
    participant_idx = participant_data['participant_idx'].iloc[0]

    print(trace.posterior['k'].mean(dim=['chain', 'draw']).shape)
    
    # Extract posterior means for this participant's parameters
    k = trace.posterior['k'].mean(dim=['chain', 'draw'])[:, participant_idx].item()
    beta = trace.posterior['beta'].mean(dim=['chain', 'draw'])[:, participant_idx].item()
    beta0 = trace.posterior['beta0'].mean(dim=['chain', 'draw'])[:, participant_idx].item()
    beta1 = trace.posterior['beta1'].mean(dim=['chain', 'draw'])[:, participant_idx].item()

    print(k, beta, beta0, beta1)
    
    # Get participant's trials
    trials = data[data['participant'] == participant_id].copy()

    # runcert / (1 + k[participant_ids] * event_prob)
    
    # Calculate utility differences
    trials['utility_cert'] = trials['rcert']
    trials['utility_uncert'] = trials['runcert'] / (1 + k * trials['event_prob'])
    
    # Predict choices using softmax
    trials['pred_choice_prob'] = 1 / (1 + np.exp(-k * trials['utility_diff']))
    trials['pred_choice'] = (trials['pred_choice_prob'] > 0.5).astype(int)
    
    # Predict RTs
    trials['pred_rt'] = beta0 + beta1 * np.abs(trials['utility_diff'])
    
    return trials[['rcert', 'runcert', 'event_prob', 'choice', 'pred_choice', 'rt', 'pred_rt']]

# Example usage:
predictions = predict_participant_behavior('bh348lli7', trace, df)
print(predictions.head())
